In [7]:
from functools import reduce
import numpy as np

#numpy
x = np.array([[1, 0, 1], [2, 0, 2], [3, 0, 3], [4, 4, 4]])
%timeit np.prod(np.diag(x)[np.diag(x)!=0])

#без numpy
x = [[1, 0, 1], [2, 0, 2], [3, 0, 3], [4, 4, 4]]
%timeit reduce(lambda x, y: x*y,[x[i][i] for i in range(len(x[0])) if x[i][i]!=0])

x = [[1, 0, 1], [2, 0, 2], [3, 0, 3], [4, 4, 4]]
def red(x):
    ret = 1
    for i in range(len(x[0])):
        if x[i][i]!=0:
            ret *= x[i][i]
    return ret
%timeit red(x)
print(red(x))

5.76 µs ± 68 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
506 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
296 ns ± 8.72 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
3


In [8]:
import numpy as np

#с numpy
x = np.array([[9, 4, 2], [6, 0, 0], [9, 9, 3]])
i = np.array([1, 2, 1])
j = np.array([1, 0, 1])
%timeit x[i, j]

#без numpy
x = [[9, 4, 2], [6, 0, 0], [9, 9, 3]]
i = [1, 2, 1]
j = [1, 0, 1]
%timeit [x[i[k]][j[k]] for k in range(len(i))]

ret = []
%timeit for k in range(len(i)): ret.append(x[i[k]][j[k]])
print([x[i[k]][j[k]] for k in range(len(i))])

1.51 µs ± 8.32 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
342 ns ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
399 ns ± 8.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
[0, 9, 0]


In [9]:
import numpy as np

#с numpy
x = np.array([1, 2, 2, 4])
y = np.array([4, 2, 1, 2])
%timeit np.array_equal(np.sort(x),np.sort(y))

#без numpy
x = [1, 2, 2, 4]
y = [4, 2, 1, 2]
%timeit x.sort()==y.sort()

def true(x, y):
    ret = []
    for i in range(len(x)):
        for j in range(len(y)):
            if x[i] == y[j]:
                ret.append(True)
                break
            elif j+1 == len(y):
                ret.append(False)
    return ret  
%timeit all(true(x,y))

4 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
85.6 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
1.18 µs ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
import numpy as np

#с numpy
x = np.array([6, 2, 0, 3, 0, 0, 5, 7, 0])
%timeit np.max(x[1:][(x==0)[:-1]])

#без numpy
x = [6, 2, 0, 3, 0, 0, 5, 7, 0]
%timeit max(x[i] for i in range(1, len(x)) if not x[i-1])

def fun(x):
    ret = []
    for i in range(1, len(x)):
        if not x[i-1]:
            ret.append(x[i])
    return ret
%timeit max(fun(x))

3.26 µs ± 36.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
605 ns ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
542 ns ± 17.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [31]:
import numpy as np
import imageio
from PIL import Image

img = Image.open('Картинка.png')

#с numpy
color = np.array([0.2989, 0.587, 0.114])
def fun(img, color):
    arr = np.asarray(img, dtype='uint8')
    new_img = Image.fromarray(np.dot(arr[...,:3], color))
    return new_img

%timeit fun(img, color)

#без numpy
%timeit img.convert("L")
color = [0.2989, 0.587, 0.114, 0]
img = imageio.imread('Картинка.png')
img = img.astype('float64')
%timeit for i in range(len(img)): img[i] += color
imageio.imwrite('Картинка-серая.png', img[:,:,0])

185 ms ± 3.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.37 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Lossy conversion from float64 to uint8. Range [39.210900000000066, 279.21090000000027]. Convert image to uint8 prior to saving to suppress this warning.


46.2 ms ± 962 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
import numpy as np
from itertools import groupby

#с numpy
x = np.array([2, 2, 2, 3, 3, 3, 5])
def fun(seq):
    ret = np.append(np.where(seq[1:] != seq[:-1]), len(seq) - 1)
    count = np.diff(np.append(-1, ret))
    return (ret,count)
%timeit fun(x)

x = [2, 2, 2, 3, 3, 3, 5]

#без numpy
def fun1(seq):
    ret = ([seq[0]],[])
    count = 1
    j = 0
    for i in range(1,len(seq)):
        if seq[i] == ret[0][j]:
            count += 1
        else :
            ret[0].append(seq[i])
            ret[1].append(count)
            j += 1
            count = 1
    ret[1].append(count)
    return ret
%timeit fun1(x)

def fun2(seq):
    ret = ([],[])
    for x, group in groupby(seq):
        ret[0].append(x)
        ret[1].append(len(tuple(group)))
    return ret
%timeit fun2(x)

28 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.08 µs ± 42.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.4 µs ± 40.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [29]:
import numpy as np
from scipy.spatial import distance
import math

#с numpy
x = np.array([2, 7, 6, 6, 9, 6, 3, 4, 9])
y = np.array([1, 0, 0, 7, 2, 2, 4, 3, 0])
%timeit np.sqrt(np.dot(x-y, x-y))

x = [2, 7, 6, 6, 9, 6, 3, 4, 9]
y = [1, 0, 0, 7, 2, 2, 4, 3, 0]
#с scipy
%timeit distance.euclidean(x, y)
#с чистым python
%timeit math.dist(x, y)
print(math.dist(x, y))

7.32 µs ± 41.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
18.6 µs ± 68.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
457 ns ± 4.31 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
15.329709716755893


In [30]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spln
# covariance matrix
sigma = np.array([[2.3, 0, 0, 0],
           [0, 1.5, 0, 0],
           [0, 0, 1.7, 0],
           [0, 0,   0, 2]
          ])
# mean vector
mu = np.array([2,3,8,10])

# input
x = np.array([2.1,3.5,8, 9.5])

def lognormpdf(x,mu,S):
    nx = len(S)
    norm_coeff = nx*np.log(2*np.pi)+np.linalg.slogdet(S)[1]
    err = x-mu
    numerator = spln.spsolve(S, err).T.dot(err)
    return -0.5*(norm_coeff+numerator)
%timeit lognormpdf(x, mu, sigma)
print (lognormpdf(x, mu, sigma))

C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


359 µs ± 2.82 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-5.054836210528194
